# Parameters
CampañasxPeriodo=18
UltimaCampañaRolling=202818
CampañaInicioRolling=202612
Carpeta=r'C:\Users\williamtorres\OneDrive - CETCO S.A\SAP IBP\Rolling forecast\Corrida 09.07.2025\SAP\Resultado CORP\103'

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
df = pd.read_csv(rf'{Carpeta}\df_NovoApp.csv')

In [4]:
df.head()

,Ce.,Material,Campaña,Fuente,Tipo de Oferta,Cantidad estimada,Descontinuación,Descripción,Grupo art.,Tipo de producto,Tipo,Grupo art..1,Crecimiento X,Crecimiento X+1,Crecimiento X+2,Crecimiento X+3,EDL
0,BO03,200118416.0,202606,Novoapp,300,2399,202616,LB HOMME EXP DEO ROLL ON 50G,103.0,DESODORANTE,103,Cuidado Personal,-0.011057,0.007899,0.003341,0.002825,-
1,BO03,200118511.0,202606,Novoapp,300,239,0,LB HOMME EXP SHAMPOO 250ML,103.0,SHAMPOO,103,Cuidado Personal,-0.011057,0.007899,0.003341,0.002825,-
2,BO03,200119532.0,202606,Novoapp,108,2354,0,CZ TRAX ROLL ON 50G,103.0,DESODORANTE,103,Cuidado Personal,-0.011057,0.007899,0.003341,0.002825,-
3,BO03,200119718.0,202606,Novoapp,108,824,0,CZ ALL BLACK ROLL ON 50G,103.0,DESODORANTE,103,Cuidado Personal,-0.011057,0.007899,0.003341,0.002825,-
4,BO03,210109608.0,202606,Novoapp,300,192,0,LB HOMME EXP ESPUMA RETY 100ML,103.0,PRODUCTOS AFEITADO-DEPILADO,103,Cuidado Personal,-0.011057,0.007899,0.003341,0.002825,-


In [5]:
# Crear tabla dinámica
tabla_pivot = pd.pivot_table(
    df,
    index=["Ce.", "Grupo art.", "Crecimiento X+2", "Crecimiento X+3", "Descontinuación", "Material"], # filas
    columns="Campaña",   # columnas
    values="Cantidad estimada",  # la métrica que quieres sumar/promediar
    aggfunc="sum",       # puede ser sum, mean, count, etc.
    fill_value=0
)

In [6]:
tabla_pivot

Campaña                                                                      202606  \
Ce.  Grupo art. Crecimiento X+2 Crecimiento X+3 Descontinuación Material              
BO03 103.0      0.003341        0.002825        0               200118472.0       0   
                                                                200118511.0     239   
                                                                200119532.0    2354   
                                                                200119718.0     824   
                                                                210109608.0     192   
...                                                                             ...   
SV13 103.0      0.003341        0.002825        0               200118511.0     433   
                                                                200119532.0    1562   
                                                                200119718.0     411   
                                                                210109608.0     582   
                                                202616          200118416.0    1565   

Campaña                                                                      202607  \
Ce.  Grupo art. Crecimiento X+2 Crecimiento X+3 Descontinuación Material              
BO03 103.0      0.003341        0.002825        0               200118472.0     153   
                                                                200118511.0      90   
                                                                200119532.0    2354   
                                                                200119718.0     824   
                                                                210109608.0     162   
...                                                                             ...   
SV13 103.0      0.003341        0.002825        0               200118511.0      72   
                                                                200119532.0    1562   
                                                                200119718.0     411   
                                                                210109608.0     529   
                                                202616          200118416.0    1391   

Campaña                                                                      202608  \
Ce.  Grupo art. Crecimiento X+2 Crecimiento X+3 Descontinuación Material              
BO03 103.0      0.003341        0.002825        0               200118472.0     153   
                                                                200118511.0       0   
                                                                200119532.0    2354   
                                                                200119718.0     824   
                                                                210109608.0      72   
...                                                                             ...   
SV13 103.0      0.003341        0.002825        0               200118511.0       0   
                                                                200119532.0    1562   
                                                                200119718.0     411   
                                                                210109608.0     379   
                                                202616          200118416.0    1054   

Campaña                                                                      202609  \
Ce.  Grupo art. Crecimiento X+2 Crecimiento X+3 Descontinuación Material              
BO03 103.0      0.003341        0.002825        0               200118472.0       0   
                                                                200118511.0       0   
                                                                200119532.0    2354   
                                                                200119718.0     824   
                                                                210109608.0       

In [7]:
def generar_campañas(inicio, fin):
    """
    Genera lista de campañas desde 'inicio' hasta 'fin'
    considerando que cada año solo llega hasta 18,
    luego pasa al siguiente año con 01.
    """
    campañas = []
    año = int(str(inicio)[:4])
    periodo = int(str(inicio)[-2:])
    año_fin = int(str(fin)[:4])
    periodo_fin = int(str(fin)[-2:])

    while (año < año_fin) or (año == año_fin and periodo <= periodo_fin):
        campañas.append(int(f"{año}{periodo:02d}"))
        periodo += 1
        if periodo > CampañasxPeriodo:   # al pasar de 18 reinicia
            periodo = 1
            año += 1
    return campañas


In [8]:
# Generar campañas desde la mínima hasta 202818
campaña_min = int(df["Campaña"].min())
campañas_completas = generar_campañas(campaña_min, UltimaCampañaRolling)

# Reindexar columnas
tabla_pivot = tabla_pivot.reindex(columns=campañas_completas, fill_value=0)

# Tabla plana opcional
tabla_pivot = tabla_pivot.reset_index()

In [9]:
tabla_pivot.head()

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202809,202810,202811,202812,202813,202814,202815,202816,202817,202818
0,BO03,103.0,0.003341,0.002825,0,200118472.0,0,153,153,0,...,0,0,0,0,0,0,0,0,0,0
1,BO03,103.0,0.003341,0.002825,0,200118511.0,239,90,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BO03,103.0,0.003341,0.002825,0,200119532.0,2354,2354,2354,2354,...,0,0,0,0,0,0,0,0,0,0
3,BO03,103.0,0.003341,0.002825,0,200119718.0,824,824,824,824,...,0,0,0,0,0,0,0,0,0,0
4,BO03,103.0,0.003341,0.002825,0,210109608.0,192,162,72,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Buscar la última campaña con valor > 0 en cada fila
tabla_pivot["ÚltimoCampañaNovoApp"] = tabla_pivot.apply(
    lambda row: max([col for col in tabla_pivot.columns if isinstance(col, int) and row[col] > 0], default=None),
    axis=1
)

In [11]:
tabla_pivot_CargadoSAP=tabla_pivot.copy()

In [12]:
tabla_pivot

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202810,202811,202812,202813,202814,202815,202816,202817,202818,ÚltimoCampañaNovoApp
0,BO03,103.0,0.003341,0.002825,0,200118472.0,0,153,153,0,...,0,0,0,0,0,0,0,0,0,202608
1,BO03,103.0,0.003341,0.002825,0,200118511.0,239,90,0,0,...,0,0,0,0,0,0,0,0,0,202607
2,BO03,103.0,0.003341,0.002825,0,200119532.0,2354,2354,2354,2354,...,0,0,0,0,0,0,0,0,0,202613
3,BO03,103.0,0.003341,0.002825,0,200119718.0,824,824,824,824,...,0,0,0,0,0,0,0,0,0,202613
4,BO03,103.0,0.003341,0.002825,0,210109608.0,192,162,72,0,...,0,0,0,0,0,0,0,0,0,202608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,SV13,103.0,0.003341,0.002825,0,200118511.0,433,72,0,0,...,0,0,0,0,0,0,0,0,0,202607
62,SV13,103.0,0.003341,0.002825,0,200119532.0,1562,1562,1562,1562,...,0,0,0,0,0,0,0,0,0,202614
63,SV13,103.0,0.003341,0.002825,0,200119718.0,411,411,411,411,...,0,0,0,0,0,0,0,0,0,202613
64,SV13,103.0,0.003341,0.002825,0,210109608.0,582,529,379,0,...,0,0,0,0,0,0,0,0,0,202608


In [13]:
def siguiente_campaña(camp):
    """Dada una campaña tipo 202615, devuelve la siguiente (202616 o 202701 si era 18)."""
    camp = int(camp)
    año = camp // 100
    num = camp % 100
    if num == 18:
        año += 1
        num = 1
    else:
        num += 1
    return int(f"{año}{num:02d}")


for idx_row, row in tabla_pivot.iterrows():
    ultima = row["ÚltimoCampañaNovoApp"]
    descontinuacion = row["Descontinuación"]

    if pd.isna(descontinuacion) or descontinuacion == 0:
        descontinuacion = UltimaCampañaRolling
    
    if pd.isna(ultima):
        continue
    
    # siguiente campaña al último estimado
    siguiente = siguiente_campaña(ultima)
    
    # inicio: mayor entre 202612 y esa siguiente
    inicio = max(CampañaInicioRolling, siguiente)

    # si no está en campañas_completas (por ejemplo 202619), saltamos al próximo válido
    while inicio not in campañas_completas and inicio < max(campañas_completas):
        inicio = siguiente_campaña(inicio)

    if inicio not in campañas_completas:
        continue  # nada que proyectar

    start_index = campañas_completas.index(inicio)

    # proyectar desde inicio hasta el final del horizonte
    for j in range(start_index, len(campañas_completas)):
        camp = campañas_completas[j]
        if camp > descontinuacion:  # 🚫 parar si pasa la campaña de descontinuación
            break
        prev_3 = campañas_completas[start_index-3:start_index]   # las 3 campañas anteriores en la lista
        tabla_pivot.at[idx_row, camp] = row[prev_3].mean()


C:\Users\williamtorres\AppData\Local\Temp\ipykernel_37824\831746151.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '382.6666666666667' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tabla_pivot.at[idx_row, camp] = row[prev_3].mean()
C:\Users\williamtorres\AppData\Local\Temp\ipykernel_37824\831746151.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '382.6666666666667' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tabla_pivot.at[idx_row, camp] = row[prev_3].mean()
C:\Users\williamtorres\AppData\Local\Temp\ipykernel_37824\831746151.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '382.6666666666667' has dtype incompatible with int64, please explicitly cast to a compatible dtype firs

In [14]:
tabla_pivot

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202810,202811,202812,202813,202814,202815,202816,202817,202818,ÚltimoCampañaNovoApp
0,BO03,103.0,0.003341,0.002825,0,200118472.0,0,153,153,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,202608
1,BO03,103.0,0.003341,0.002825,0,200118511.0,239,90,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,202607
2,BO03,103.0,0.003341,0.002825,0,200119532.0,2354,2354,2354,2354,...,2354.0,2354.0,2354.0,2354.0,2354.0,2354.0,2354.0,2354.0,2354.0,202613
3,BO03,103.0,0.003341,0.002825,0,200119718.0,824,824,824,824,...,824.0,824.0,824.0,824.0,824.0,824.0,824.0,824.0,824.0,202613
4,BO03,103.0,0.003341,0.002825,0,210109608.0,192,162,72,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,202608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,SV13,103.0,0.003341,0.002825,0,200118511.0,433,72,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,202607
62,SV13,103.0,0.003341,0.002825,0,200119532.0,1562,1562,1562,1562,...,1562.0,1562.0,1562.0,1562.0,1562.0,1562.0,1562.0,1562.0,1562.0,202614
63,SV13,103.0,0.003341,0.002825,0,200119718.0,411,411,411,411,...,411.0,411.0,411.0,411.0,411.0,411.0,411.0,411.0,411.0,202613
64,SV13,103.0,0.003341,0.002825,0,210109608.0,582,529,379,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,202608


In [15]:
año_base = datetime.today().year

for idx_row, row in tabla_pivot.iterrows():
    crec_x2 = row["Crecimiento X+2"]  if not pd.isna(row["Crecimiento X+2"]) else 0
    crec_x3 = row["Crecimiento X+3"]  if not pd.isna(row["Crecimiento X+3"]) else 0

    for camp in campañas_completas:
        año = int(str(camp)[:4])  # los 4 primeros dígitos de la campaña

        if año == año_base + 2:
            tabla_pivot.at[idx_row, camp] = tabla_pivot.at[idx_row, camp] * (1 + crec_x2)

        elif año == año_base + 3:
            tabla_pivot.at[idx_row, camp] = tabla_pivot.at[idx_row, camp] * (1 + crec_x3+crec_x2)


In [16]:
# Detectar columnas que son campañas (numéricas tipo 202501, 202502, etc.)
camp_cols = [c for c in tabla_pivot.columns if str(c).isdigit()]
tabla_pivot = tabla_pivot.fillna(0)
# Redondear hacia arriba sin decimales
tabla_pivot[camp_cols] = np.ceil(tabla_pivot[camp_cols]).astype(int)

In [17]:
tabla_pivot

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202810,202811,202812,202813,202814,202815,202816,202817,202818,ÚltimoCampañaNovoApp
0,BO03,103.0,0.003341,0.002825,0,200118472.0,0,153,153,0,...,0,0,0,0,0,0,0,0,0,202608
1,BO03,103.0,0.003341,0.002825,0,200118511.0,239,90,0,0,...,0,0,0,0,0,0,0,0,0,202607
2,BO03,103.0,0.003341,0.002825,0,200119532.0,2354,2354,2354,2354,...,2369,2369,2369,2369,2369,2369,2369,2369,2369,202613
3,BO03,103.0,0.003341,0.002825,0,200119718.0,824,824,824,824,...,830,830,830,830,830,830,830,830,830,202613
4,BO03,103.0,0.003341,0.002825,0,210109608.0,192,162,72,0,...,0,0,0,0,0,0,0,0,0,202608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,SV13,103.0,0.003341,0.002825,0,200118511.0,433,72,0,0,...,0,0,0,0,0,0,0,0,0,202607
62,SV13,103.0,0.003341,0.002825,0,200119532.0,1562,1562,1562,1562,...,1572,1572,1572,1572,1572,1572,1572,1572,1572,202614
63,SV13,103.0,0.003341,0.002825,0,200119718.0,411,411,411,411,...,414,414,414,414,414,414,414,414,414,202613
64,SV13,103.0,0.003341,0.002825,0,210109608.0,582,529,379,0,...,0,0,0,0,0,0,0,0,0,202608


In [18]:
# Identificar columnas de campañas (numéricas tipo 202501, 202502, etc.)
camp_cols = [c for c in tabla_pivot.columns if str(c).isdigit()]

# Copia base
tabla_proyecciones = tabla_pivot.copy()

for idx_row, row in tabla_pivot.iterrows():
    ultima = int(row["ÚltimoCampañaNovoApp"])
    # Todo lo que sea <= a la última campaña lo borramos (NaN) → solo dejamos proyecciones
    for col in camp_cols:
        if int(col) <= ultima:
            tabla_proyecciones.at[idx_row, col] = np.nan


In [19]:
# Identificar columnas de campañas (numéricas tipo 202501, 202502, etc.)
camp_cols = [c for c in tabla_pivot.columns if str(c).isdigit()]

# Copia base
tabla_proyecciones_Sistema = tabla_pivot.copy()



In [20]:
tabla_proyecciones

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202810,202811,202812,202813,202814,202815,202816,202817,202818,ÚltimoCampañaNovoApp
0,BO03,103.0,0.003341,0.002825,0,200118472.0,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,202608
1,BO03,103.0,0.003341,0.002825,0,200118511.0,NaN,NaN,0.0,0.0,...,0,0,0,0,0,0,0,0,0,202607
2,BO03,103.0,0.003341,0.002825,0,200119532.0,NaN,NaN,NaN,NaN,...,2369,2369,2369,2369,2369,2369,2369,2369,2369,202613
3,BO03,103.0,0.003341,0.002825,0,200119718.0,NaN,NaN,NaN,NaN,...,830,830,830,830,830,830,830,830,830,202613
4,BO03,103.0,0.003341,0.002825,0,210109608.0,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,202608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,SV13,103.0,0.003341,0.002825,0,200118511.0,NaN,NaN,0.0,0.0,...,0,0,0,0,0,0,0,0,0,202607
62,SV13,103.0,0.003341,0.002825,0,200119532.0,NaN,NaN,NaN,NaN,...,1572,1572,1572,1572,1572,1572,1572,1572,1572,202614
63,SV13,103.0,0.003341,0.002825,0,200119718.0,NaN,NaN,NaN,NaN,...,414,414,414,414,414,414,414,414,414,202613
64,SV13,103.0,0.003341,0.002825,0,210109608.0,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,202608


In [21]:
tabla_proyecciones.to_excel(
    rf"{Carpeta}\ProyecionNovoAPP.xlsx", 
    sheet_name="Proyeccion", 
    index=False
)

In [22]:
# Detectar columnas que son campañas (numéricas tipo 202501, 202612, etc.)
camp_cols = [c for c in tabla_proyecciones.columns if str(c).isdigit()]

# Filtrar solo las columnas de campañas >= 202612
camp_cols_filtradas = [c for c in camp_cols if int(c) >= CampañaInicioRolling]

# Mantener columnas fijas + las filtradas
otras_cols = [c for c in tabla_proyecciones.columns if c not in camp_cols]
tabla_filtrada = tabla_proyecciones[otras_cols + camp_cols_filtradas]


In [23]:
tabla_filtrada = tabla_filtrada.fillna(0)
tabla_filtrada

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,ÚltimoCampañaNovoApp,202612,202613,202614,...,202809,202810,202811,202812,202813,202814,202815,202816,202817,202818
0,BO03,103.0,0.003341,0.002825,0,200118472.0,202608,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,BO03,103.0,0.003341,0.002825,0,200118511.0,202607,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,BO03,103.0,0.003341,0.002825,0,200119532.0,202613,0.0,0.0,2354.0,...,2369,2369,2369,2369,2369,2369,2369,2369,2369,2369
3,BO03,103.0,0.003341,0.002825,0,200119718.0,202613,0.0,0.0,824.0,...,830,830,830,830,830,830,830,830,830,830
4,BO03,103.0,0.003341,0.002825,0,210109608.0,202608,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,SV13,103.0,0.003341,0.002825,0,200118511.0,202607,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
62,SV13,103.0,0.003341,0.002825,0,200119532.0,202614,0.0,0.0,0.0,...,1572,1572,1572,1572,1572,1572,1572,1572,1572,1572
63,SV13,103.0,0.003341,0.002825,0,200119718.0,202613,0.0,0.0,411.0,...,414,414,414,414,414,414,414,414,414,414
64,SV13,103.0,0.003341,0.002825,0,210109608.0,202608,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
tabla_filtrada.columns

Index([                 'Ce.',           'Grupo art.',      'Crecimiento X+2',
            'Crecimiento X+3',      'Descontinuación',             'Material',
       'ÚltimoCampañaNovoApp',                 202612,                 202613,
                       202614,                 202615,                 202616,
                       202617,                 202618,                 202701,
                       202702,                 202703,                 202704,
                       202705,                 202706,                 202707,
                       202708,                 202709,                 202710,
                       202711,                 202712,                 202713,
                       202714,                 202715,                 202716,
                       202717,                 202718,                 202801,
                       202802,                 202803,                 202804,
                       202805,                 20280

In [25]:
tabla_filtrada= tabla_filtrada.drop(columns=['Grupo art.','Crecimiento X+2',
            'Crecimiento X+3',      'Descontinuación', 'ÚltimoCampañaNovoApp'])
tabla_filtrada.head()

Campaña,Ce.,Material,202612,202613,202614,202615,202616,202617,202618,202701,...,202809,202810,202811,202812,202813,202814,202815,202816,202817,202818
0,BO03,200118472.0,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BO03,200118511.0,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BO03,200119532.0,0.0,0.0,2354.0,2354.0,2354.0,2354,2354,2362,...,2369,2369,2369,2369,2369,2369,2369,2369,2369,2369
3,BO03,200119718.0,0.0,0.0,824.0,824.0,824.0,824,824,827,...,830,830,830,830,830,830,830,830,830,830
4,BO03,210109608.0,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
tabla_larga = tabla_filtrada.melt(
    id_vars=["Ce.", 'Material'],  
    var_name="Campaña",
    value_name="Estimado"
)

In [27]:
tabla_larga=tabla_larga[tabla_larga["Estimado"]>0]
tabla_larga["TO"] = "'000"
tabla_larga["Estimado"] = tabla_larga["Estimado"].astype(int)
tabla_larga["Material"] = tabla_larga["Material"].astype(int)
tabla_larga.columns=["Ce.",'Cod', 'Campaña', 'Estimado', "TO" ]
tabla_larga.head()

,Ce.,Cod,Campaña,Estimado,TO
37,GT23,200118511,202612,653,'000
48,PA33,200118472,202612,118,'000
49,PA33,200118511,202612,241,'000
103,GT23,200118511,202613,653,'000
114,PA33,200118472,202613,118,'000


In [28]:
tabla_larga=tabla_larga[["Ce.", 'Campaña', 'Cod', "TO", 'Estimado']]

In [29]:
tabla_larga

,Ce.,Campaña,Cod,TO,Estimado
37,GT23,202612,200118511,'000,653
48,PA33,202612,200118472,'000,118
49,PA33,202612,200118511,'000,241
103,GT23,202613,200118511,'000,653
114,PA33,202613,200118472,'000,118
...,...,...,...,...,...
2824,PA33,202818,210109608,'000,168
2828,PE03,202818,200119532,'000,7257
2829,PE03,202818,200119718,'000,3443
2834,SV13,202818,200119532,'000,1572


In [30]:
tabla_larga.to_excel(
    rf"{Carpeta}\CargaNovoApp.xlsx", 
    sheet_name="Proyeccion", 
    index=False
)

In [31]:
tabla_larga["Año"] = tabla_larga["Campaña"].astype(str).str[:4].astype(int)
tabla_larga["campaña"]   = tabla_larga["Campaña"].astype(str).str[-2:].astype(int)

In [32]:
tabla_larga

,Ce.,Campaña,Cod,TO,Estimado,Año,campaña
37,GT23,202612,200118511,'000,653,2026,12
48,PA33,202612,200118472,'000,118,2026,12
49,PA33,202612,200118511,'000,241,2026,12
103,GT23,202613,200118511,'000,653,2026,13
114,PA33,202613,200118472,'000,118,2026,13
...,...,...,...,...,...,...,...
2824,PA33,202818,210109608,'000,168,2028,18
2828,PE03,202818,200119532,'000,7257,2028,18
2829,PE03,202818,200119718,'000,3443,2028,18
2834,SV13,202818,200119532,'000,1572,2028,18


In [33]:
tabla_grouped = (
    tabla_larga
    .groupby(["Año", "campaña"], as_index=False)["Estimado"]
    .sum()
)

In [34]:
tabla_pivot = tabla_larga.pivot_table(
    index="campaña",        # filas
    columns="Año",    # columnas
    values="Estimado",      # valores
    aggfunc="sum",    # sumar
    fill_value=0      # reemplazar NaN con 0
)


# Quitar el nombre del índice de columnas ("Año")
tabla_pivot.columns.name = None

# Opcional: resetear índice si no quieres que "C" quede como índice
tabla_pivot = tabla_pivot.reset_index()

## Estimados cargados por Rolling

In [35]:
tabla_pivot

,campaña,2026,2027,2028
0,1,0,70385,70584
1,2,0,70385,70584
2,3,0,70385,70584
3,4,0,70385,70584
4,5,0,70385,70584
5,6,0,70385,70584
6,7,0,70385,70584
7,8,0,70385,70584
8,9,0,70385,70584
9,10,0,70385,70584


In [36]:
tabla_pivot.to_csv(rf"{Carpeta}\TotalNovoAPPRolling.csv", index=False, encoding="utf-8-sig")

## Total NovoApp con propagación

In [37]:
tabla_proyecciones=tabla_proyecciones_Sistema

In [38]:
# Detectar columnas que son campañas (numéricas tipo 202501, 202612, etc.)
camp_cols = [c for c in tabla_proyecciones.columns if str(c).isdigit()]

# Filtrar solo las columnas de campañas >= 202612
camp_cols_filtradas = [c for c in camp_cols if int(c) >= CampañaInicioRolling]

# Mantener columnas fijas + las filtradas
otras_cols = [c for c in tabla_proyecciones.columns if c not in camp_cols]
tabla_filtrada = tabla_proyecciones[otras_cols + camp_cols_filtradas]
tabla_filtrada = tabla_filtrada.fillna(0)
tabla_filtrada
tabla_filtrada= tabla_filtrada.drop(columns=['Grupo art.','Crecimiento X+2',
            'Crecimiento X+3',      'Descontinuación', 'ÚltimoCampañaNovoApp'])
tabla_larga = tabla_filtrada.melt(
    id_vars=["Ce.", 'Material'],  
    var_name="Campaña",
    value_name="Estimado"
)
tabla_larga=tabla_larga[tabla_larga["Estimado"]>0]
tabla_larga["TO"] = "'000"
tabla_larga["Estimado"] = tabla_larga["Estimado"].astype(int)
tabla_larga["Material"] = tabla_larga["Material"].astype(int)
tabla_larga.columns=["Ce.",'Cod', 'Campaña', 'Estimado', "TO" ]
tabla_larga=tabla_larga[["Ce.", 'Campaña', 'Cod', "TO", 'Estimado']]
tabla_larga["Año"] = tabla_larga["Campaña"].astype(str).str[:4].astype(int)
tabla_larga["campaña"]   = tabla_larga["Campaña"].astype(str).str[-2:].astype(int)
tabla_grouped = (
    tabla_larga
    .groupby(["Año", "campaña"], as_index=False)["Estimado"]
    .sum()
)
tabla_pivot = tabla_larga.pivot_table(
    index="campaña",        # filas
    columns="Año",    # columnas
    values="Estimado",      # valores
    aggfunc="sum",    # sumar
    fill_value=0      # reemplazar NaN con 0
)


# Quitar el nombre del índice de columnas ("Año")
tabla_pivot.columns.name = None

# Opcional: resetear índice si no quieres que "C" quede como índice
tabla_pivot = tabla_pivot.reset_index()

In [39]:
tabla_pivot

,campaña,2026,2027,2028
0,1,0,70385,70584
1,2,0,70385,70584
2,3,0,70385,70584
3,4,0,70385,70584
4,5,0,70385,70584
5,6,0,70385,70584
6,7,0,70385,70584
7,8,0,70385,70584
8,9,0,70385,70584
9,10,0,70385,70584


In [40]:
tabla_pivot.to_csv(rf"{Carpeta}\TotalNovoAPP.csv", index=False, encoding="utf-8-sig")

## Cargados en el sistema

In [41]:
tabla_pivot_CargadoSAP

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202810,202811,202812,202813,202814,202815,202816,202817,202818,ÚltimoCampañaNovoApp
0,BO03,103.0,0.003341,0.002825,0,200118472.0,0,153,153,0,...,0,0,0,0,0,0,0,0,0,202608
1,BO03,103.0,0.003341,0.002825,0,200118511.0,239,90,0,0,...,0,0,0,0,0,0,0,0,0,202607
2,BO03,103.0,0.003341,0.002825,0,200119532.0,2354,2354,2354,2354,...,0,0,0,0,0,0,0,0,0,202613
3,BO03,103.0,0.003341,0.002825,0,200119718.0,824,824,824,824,...,0,0,0,0,0,0,0,0,0,202613
4,BO03,103.0,0.003341,0.002825,0,210109608.0,192,162,72,0,...,0,0,0,0,0,0,0,0,0,202608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,SV13,103.0,0.003341,0.002825,0,200118511.0,433,72,0,0,...,0,0,0,0,0,0,0,0,0,202607
62,SV13,103.0,0.003341,0.002825,0,200119532.0,1562,1562,1562,1562,...,0,0,0,0,0,0,0,0,0,202614
63,SV13,103.0,0.003341,0.002825,0,200119718.0,411,411,411,411,...,0,0,0,0,0,0,0,0,0,202613
64,SV13,103.0,0.003341,0.002825,0,210109608.0,582,529,379,0,...,0,0,0,0,0,0,0,0,0,202608


In [42]:
tabla_proyecciones=tabla_pivot_CargadoSAP

In [43]:
# Detectar columnas que son campañas (numéricas tipo 202501, 202612, etc.)
camp_cols = [c for c in tabla_proyecciones.columns if str(c).isdigit()]

# Filtrar solo las columnas de campañas >= 202612
camp_cols_filtradas = [c for c in camp_cols if int(c) >= CampañaInicioRolling]

# Mantener columnas fijas + las filtradas
otras_cols = [c for c in tabla_proyecciones.columns if c not in camp_cols]
tabla_filtrada = tabla_proyecciones[otras_cols + camp_cols_filtradas]
tabla_filtrada = tabla_filtrada.fillna(0)
tabla_filtrada
tabla_filtrada= tabla_filtrada.drop(columns=['Grupo art.','Crecimiento X+2',
            'Crecimiento X+3',      'Descontinuación', 'ÚltimoCampañaNovoApp'])
tabla_larga = tabla_filtrada.melt(
    id_vars=["Ce.", 'Material'],  
    var_name="Campaña",
    value_name="Estimado"
)
#tabla_larga=tabla_larga[tabla_larga["Estimado"]>0]
tabla_larga["TO"] = "'000"
tabla_larga["Estimado"] = tabla_larga["Estimado"].astype(int)
tabla_larga["Material"] = tabla_larga["Material"].astype(int)
tabla_larga.columns=["Ce.",'Cod', 'Campaña', 'Estimado', "TO" ]
tabla_larga=tabla_larga[["Ce.", 'Campaña', 'Cod', "TO", 'Estimado']]
tabla_larga["Año"] = tabla_larga["Campaña"].astype(str).str[:4].astype(int)
tabla_larga["campaña"]   = tabla_larga["Campaña"].astype(str).str[-2:].astype(int)
tabla_grouped = (
    tabla_larga
    .groupby(["Año", "campaña"], as_index=False)["Estimado"]
    .sum()
)
tabla_pivot = tabla_larga.pivot_table(
    index="campaña",        # filas
    columns="Año",    # columnas
    values="Estimado",      # valores
    aggfunc="sum",    # sumar
    fill_value=0      # reemplazar NaN con 0
)


# Quitar el nombre del índice de columnas ("Año")
tabla_pivot.columns.name = None

# Opcional: resetear índice si no quieres que "C" quede como índice
tabla_pivot = tabla_pivot.reset_index()

In [44]:
tabla_pivot

,campaña,2026,2027,2028
0,1,0,0,0
1,2,0,0,0
2,3,0,0,0
3,4,0,0,0
4,5,0,0,0
5,6,0,0,0
6,7,0,0,0
7,8,0,0,0
8,9,0,0,0
9,10,0,0,0


In [45]:
tabla_pivot.to_csv(rf"{Carpeta}\TotalNovoAPPSistema.csv", index=False, encoding="utf-8-sig")